# Lab 6: Estimating Causal Effects via Instrumental Variables
Welcome to lab seven! 

The goals of this lab is to implement and get better understanding of the two-stage least squares procedure discussed in section.

The code you need to write is commented out with a message "TODO: fill in".


## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** by adding a cell below.

**Submission**: to submit this assignment, rerun the notebook from scratch (by selecting Kernel > Restart & Run all), and then print as a pdf (File > download as > pdf) and submit it to Gradescope.


**This assignment should be completed and submitted before Friday Nov 1, 2019 at 11:59 PM.**

## Instrumental Variables Background

Suppose that we measure a set of features $\{X^*,X\}$, which affect an observed target outcome $Y$, for multiple different units. We are interested in finding the average causal effect of $X^*$ on $Y$. The effect we are interested in is **causal** because we want to know how $Y$ changes if all randomness other than $X^*$ remains fixed, and only $X^*$ changes. We will refer to $X^*$ as the "treatment." In general, $X$ might be multi-dimensional, however for the purpose of this exercise we take $X^*, X\in\mathbb{R}$.

We assume that the outcome is generated as a linear function of the feature $X$ and treatment $X^*$, with additive noise $\xi_Y$:
$$$$
$$Y = \alpha X^* + \beta X + \xi_Y.$$

If the treatment is uncorrelated with the additive noise, ordinary least squares (OLS) yields unbiased results, giving $\alpha$ in expectation. However, the treatment is often correlated with the noise; this is often due to the effect of some variables which are important for the generation of $Y$, but were not measured.

One way to get around this issue is by using instrumental variables (IVs). A valid instrument $Z$ is a variable which is independent of $\xi_Y$, and affects $Y$ only through $X^*$. Then, one way to estimate $\alpha$ is to first "guess" $X^*$ from $\{Z,X\}$ (denoted $\hat X^*$), and then regress $Y$ onto $\{\hat X^*, X\}$ (instead of $\{X^*, X\}$). If both the initial "guess" $\hat X^*$ and $Y$ regressed onto the $\{\hat X^*, X\}$ are obtained using ordinary least squares, this procedure is known as two-stage least squares (2SLS).

For a concrete example, suppose a researcher wishes to estimate the causal effect of drinking tea on fever reduction (only among sick patients, of course, as there is no need for fever reduction among healthy individuals). Correlation between the amount of tea a patient drinks and fever reduction does not necessarily imply that drinking tea improves health, because other variables, such as how long the patient has been sick and how much medication they are taking, might relate to both one's health and the amount of consumed tea. However, the researcher may attempt to estimate the causal effect of drinking tea on health by using instrumental variables. Suppose that drinking tea is sufficiently correlated with the outside temperature. Then, the researcher can use outside temperature as an instrumental variable, as it is fairly reasonable to assume that it doesn't directly affect fever reduction (but only through tea consumption).

Suppose that we have historical data from 10,000 different individuals. We have the following variables: $X^* =$ amount of consumed tea, $X =$ medication taken, $Z = $ outside temperature and $Y = $ degrees of fever reduction. Suppose that the number of sick days the patient has been sick $W$ affects all of $X^*, X, Y$, but is not measured.

In particular, suppose the data is generated as follows:
$$W \sim D_{W}, ~ ~ Z\sim D_Z,$$ (sick days and outside temperature randomly drawn from some distributions)
$$X = \delta_1 W + \xi_X,$$ (amount of medication taken is linear in the number of sick days, with random independent noise added)
$$X^* = \delta_2 W + \gamma Z + \xi_*,$$
(amount of consumed tea is linear in the number of sick days and outside temperature, with random independent noise added)
$$Y = \alpha X^* + \beta_1 W + \beta_2 X.$$ (fever reduction is linear in tea consumption, sick days and medication taken)

The true causal effect of tea consumption on fever reduction is thus $\alpha$.

We set the above parameters:

In [ ]:
import numpy as np
import statsmodels.api as sm

In [ ]:
N = 10000 # sample size
alpha = 50 # tea to fever_reduction
true_effect = alpha
gamma = -1.5 # temperature to tea
delta = 2 # days_sick to tea
beta_1 = 100 # days_sick to fever_reduction
beta_2 = -10 # medication to fever_reduction

days_sick = np.random.normal(30, 5, (N, 1)) # distribution of sick days
temperature = np.random.normal(20, 1, (N, 1)) # distribution of outside temperature
medication = np.random.normal(100, 20, (N, 1)) # distribution of taken medication

In [ ]:
tea = gamma * temperature + delta * days_sick + np.random.normal(15,5,(N,1))

In [ ]:
fever_reduction = alpha * tea + beta_1 * days_sick + beta_2 * medication

The number of sick days is unfortunately unobserved. Suppose we first estimate the causal effect using plain linear regression (OLS), where we regress fever reduction onto medication taken and tea consumption.

In [ ]:
ols_features = # TODO: make a feature matrix of tea and medication
ols_features_w_const = sm.add_constant(ols_features) # prepend a constant feature for intercept term
ols_model = sm.OLS(fever_reduction, ols_features_w_const)
ols_results = (ols_model.fit()).params # predicted coefficients
alpha_ols = # TODO: estimate of alpha

As the error, we take the absolute error $|\alpha - \alpha_{OLS}|$.

In [ ]:
# Compute Error
ols_error = # TODO: absolute error
print("Error in causal estimate of linear regression (OLS): {:.5f}".format(ols_error))

To eliminate the bias, we turn to instrumental variables. In the first stage, we "predict" tea consumption from the outside temperature and medication taken. Then, in the second stage, we regress fever reduction onto medication taken and the predicted tea consumption.

In [ ]:
# First Stage
stage1_features = # TODO
stage1_features_w_const = sm.add_constant(stage1_features) # prepend a constant feature for intercept term
stage1_ols = sm.OLS(tea, stage1_features_w_const)
stage1_ols_results = (stage1_ols.fit()).params # predicted coefficients

Now we predict tea $\hat X^*$ from $Z$ and $X$, using the model from the first stage.

In [ ]:
tea_predicted = # TODO

Below we implement the second stage, in which we regress $Y$ onto $\hat X^*$ and $X$.

In [ ]:
# Second Stage
stage2_features = # TODO
stage2_features_w_const = sm.add_constant(stage2_features)  # prepend a constant feature for intercept term
stage2_model = sm.OLS(fever_reduction, stage2_features_w_const)
stage2ols_results = (stage2_model.fit()).params
alpha_2SLS = # TODO

Again we look at the absolute error $|\alpha - \alpha_{2SLS}|$.

In [ ]:
# Compute Error
_2SLS_error = # TODO
print("Error in causal estimate of two-stage least squares (2SLS): {:.5f}".format(_2SLS_error))

### Summarize your findings below.